**Inclass material for Week 4: SQL Query using `pandas`**

This notebook was made based on main materials `4_SQL_Query.ipynb`


---

# SQL Query using `pandas`

**Training Objectives**

- Querying from SQL Databases
- SQL Joins
- SQL Conditional Statements
- Flavors and Common Operators
- End to end data analysis

# Working with SQL Databases

## Database Schema

Database `chinook.db` terdiri dari 11 tabel yang berisi sampel data dari sebuah toko yang menjual media digital:

- Data terkait media seperti pada tabel `tracks`, `albums`, `artists`, `genres`, `media_types`, `playlists`, `playlist_track` merupakan data asli dari library Apple iTunes.
- Informasi mengenai `customers` dan `employees` dibuat menggunakan nama dan alamat fiktif yang dapat ditemukan di Google maps, dan data lain yang diformat dengan baik (telepon, fax, email, dll).
- Informasi penjualan `invoices` dan `invoice_items` dihasilkan secara otomatis menggunakan data acak untuk periode tahun 2009-2013.

Skema berikut sering disebut sebagai **Entity Relationship Diagram (ERD)**, menunjukkan:
1. Entitas (Tabel)
2. Atribut (Kolom beserta tipe datanya)
3. Kardinalitas (Hubungan antar tabel)

![](assets/chinookschema2.png)

Gambar di atas adalah ERD untuk database chinook:

- **Satu database** terdiri dari **beberapa tabel**
- **Setiap tabel** terdiri dari beberapa **atribut/kolom**
- Terdapat kolom unik yang membedakan satu tabel dengan lainnya, disebut sebagai **Primary Key (PK)**. PK dapat digunakan sebagai kolom penghubung antar tabel saat dilakukan proses join (penggabungan) tabel.

Sebelum kita dapat melakukan pembacaan data, kita perlu menghubungkan Python dengan database yang ingin kita akses. Tiap koneksi database memerlukan library yang berbeda, tergantung database tersebut kompatibel dengan DBMS apa.

<br>

<details>
    <summary>✨ Connecting to MySQL</summary>
    
```python
import pymysql
  
conn = pymysql.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    db = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Oracle</summary>
    
```python
import cx_Oracle
  
# data source name from tnsnames.ora file
dsn_tns = cx_Oracle.makedsn(
    HOST_NAME,
    PORT_NUMBER
    service_name = SERVICE_NAME)

# connection
conn = cx_Oracle.connect(
    user = USER_NAME,
    password = PASSWORD,
    dsn = dsn_tns)
```
</details>

<br>

<details>
    <summary>✨ Connecting to PostgreSQL</summary>
    
```python
import psycopg2

conn = psycopg2.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    database = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Microsoft SQL Server</summary>
    
```python
import pyodbc 
conn = pyodbc.connect(
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=host;'
    'PORT=1433;'
    'UID=user;'
    'PWD=password;'
    'Database=database;')
```
</details>

Pada materi kali ini, kita akan menggunakan database _chinook_ yang kompatibel dengan SQLite, sehingga kita akan menggunakan library `sqlite3`. Library `sqlite3` merupakan library bawaan dari Python.

In [1]:
import sqlite3 
import pandas as pd

In [2]:
# membuat object conn untuk connection
conn = sqlite3.connect("data_input/chinook.db")
conn

## `SELECT` Statements

`SELECT` digunakan untuk memilih **kolom** dari sebuah **tabel**.

Syntax `SELECT`:

```sql
SELECT <NAMA_KOLOM>
FROM <NAMA_TABLE>
```

❓ Misal, hanya ingin mengambil kolom `AlbumId` dan `Title` dari tabel `albums`

In [3]:
# query
query = "SELECT AlbumId, Title FROM albums"

Untuk membaca data pada database dengan SQL pada `pandas`, kita dapat menggunakan `pd.read_sql_query()` dan menyertakan connection yang telah dibuat:

```python
pd.read_sql_query(sql, con)
```
Parameter:
- `sql`: perintah query
- `con`: connection

In [4]:
# read sql query
pd.read_sql_query(sql = query, con=conn)

,AlbumId,Title
0,1,For Those About To Rock We Salute You
1,2,Balls to the Wall
2,3,Restless and Wild
3,4,Let There Be Rock
4,5,Big Ones
...,...,...
342,343,Respighi:Pines of Rome
343,344,Schubert: The Late String Quartets & String Qu...
344,345,Monteverdi: L'Orfeo
345,346,Mozart: Chamber Music


❓ Bagaimana jika kita ingin mengambil **semua kolom** dari tabel `albums`? Gunakan command `SELECT *` 

In [5]:
# query
query = "SELECT * FROM albums"

# read sql query
pd.read_sql_query(query, conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [6]:
# sama dengan code di atas, query langsung ke parameter sql
pd.read_sql_query("SELECT * FROM albums", conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


💡 **Note**: Berbeda dari Python, perintah yang ada di SQL bersifat case **insensitive**. Query di atas dapat juga dituliskan sebagai `select * from albums`. Namun, demi kemudahan pembacaan, statement biasa dituliskan dengan huruf besar (dalam hal ini `SELECT` dan `FROM`).

In [7]:
# code here
pd.read_sql_query("SeLeCT * FRoM Albums", conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


### ✅ Knowledge Check

Kali ini, coba ambil semua kolom pada tabel `artists` kemudian simpan ke dalam sebuah object dataframe bernama `df_artist`.

**Pertanyaan:** Berapa baris & kolom yang ada pada dataframe `df_artist`?

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
```python
___ = pd.read_sql_query("SELECT ___ FROM ___", conn)
```
</details>

In [8]:
# your code here
df_artist = pd.read_sql_query("SELECT * FROM artists", conn)
df_artist.shape

(275, 2)

Berikut adalah parameter yang sering digunakan dalam method `pd.read_sql_query()`:

- `sql`: SQL query dalam bentuk string
- `con`: object connection
- `index_col`: nama atau index kolom yang ingin dijadikan index (seperti pada `pd.read_csv()`)
- `parse_dates`: nama kolom yang ingin dikonversi menjadi tipe data `datetime64[ns]` (seperti pada `pd.read_csv()`)

> Kunjungi [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) untuk detail lebih lanjut.

### `LIMIT` Statement

Statement `LIMIT` digunakan untuk mengambil beberapa baris pertama pada data.

❓ Ambil **5 baris pertama** dari tabel `artists` lalu jadikan kolom `ArtistId` sebagai index baris dengan parameter `index_col`.

In [9]:
# code here
pd.read_sql_query("SELECT * FROM artists LIMIT 5", conn, index_col='ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


💬 **Discussion :** Apakah perbedaan statement `LIMIT` dengan `head()` di pandas dataframe?

In [10]:
# code here
pd.read_sql_query("SELECT * FROM artists", conn, index_col='ArtistId').head()

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


🧐 Ambil 10 baris pertama dengan statement `LIMIT` dari tabel `albums` dan jadikan kolom `AlbumId` sebagai index

In [11]:
# code here
pd.read_sql_query('SELECT * FROM albums LIMIT 10', conn, index_col='AlbumId')

,Title,ArtistId
AlbumId,,
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4
7,Facelift,5
8,Warner 25 Anos,6
9,Plays Metallica By Four Cellos,7


## `JOIN` Statement

Statement `JOIN` digunakan untuk **menggabungkan dua tabel** menjadi satu tabel melalui **kolom penghubung** yang sama. Operasi `JOIN` dapat dilakukan lebih dari satu kali dalam satu SQL query. Berikut adalah diagram ilustrasi beberapa macam operasi SQL `JOIN`:

![](assets/sqljoins.png)

Credit: Data & Object Factory, LLC

- `LEFT JOIN` paling sering digunakan pada sebagian besar skenario bisnis. `LEFT JOIN` mengembalikan semua baris pada tabel kiri **terlepas** dari apakah ada baris yang cocok pada tabel kanan.
- `INNER JOIN` adalah tipe join yang sangat intuitif dan mudah dipahami. Query ini mengembalikan semua baris di tabel kiri yang cocok dengan tabel kanan.

**Note**:

- `RIGHT JOIN` hampir tidak pernah digunakan karena ekuivalen dengan `LEFT JOIN` hanya mengganti peletakkan tabel kiri dan kanannya saja.
- `FULL OUTER JOIN` sangat jarang digunakan. Selain alasan komputasi, tipe join ini mengembalikan semua baris dari kedua tabel terlepas dari apakah ada kecocokan atau tidak, sehingga menghasilkan DataFrame dengan banyak missing value.

Ilustrasi terkait ke-empat tipe join dapat dilihat pada [Visual JOIN](https://joins.spathon.com/).

Syntax `JOIN`:

```sql
SELECT <COLUMNS>
FROM <LEFT_TABLE>
[LEFT|INNER] JOIN <RIGHT_TABLE>
ON <LEFT_TABLE>.key = <RIGHT_TABLE>.key
```

Perhatikan kembali skema database `chinook.db`:

1. Tabel `albums`: 
    - `AlbumId`
    - `Title`
    - `ArtistId`


2. Tabel `artists`:
    - `ArtistId`
    - `Name` 
    
**Kasus:** Kita ingin mendapatkan DataFrame yang menampilkan semua baris dari kolom `AlbumId` dan `Title` secara lengkap, beserta kolom `Name` (tidak harus lengkap). Perhatikan bahwa kolom `Name` terdapat pada tabel `artists`, sedangkan kolom `AlbumId` dan `Title` terdapat pada tabel `albums`.

- Tabel kiri: albums
- Tabel kanan: artists

In [12]:
# code here
pd.read_sql_query(
    "SELECT albums.AlbumId, albums.Title, artists.Name\
    FROM albums\
    LEFT JOIN artists\
    ON albums.ArtistId = artists.ArtistId",
    conn
)

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith
...,...,...,...
342,343,Respighi:Pines of Rome,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,Nash Ensemble


Perhatikan SQL query di atas:
- Tabel `albums` menjadi tabel kiri, sedangkan tabel `artists` menjadi tabel kanan.
- Kedua tabel digabungkan menggunakan operasi `LEFT JOIN` melalui key `ArtistId`.
- Penggunaan karakter backslash (`\`) ditujukan agar ke-empat baris SQL query dibaca sebagai satu kesatuan baris. Namun penggunaan backslash kurang efisien dan memiliki resiko error yang lebih tinggi. **Sebagai alternatif, Anda dapat mengapit SQL query dengan `"""` (petik tiga kali)**

In [13]:
# code here
pd.read_sql_query(
    """
    SELECT albums.AlbumId, albums.Title, artists.Name
    FROM albums
    LEFT JOIN artists
    ON albums.ArtistId = artists.ArtistId
    """,
    conn
)

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith
...,...,...,...
342,343,Respighi:Pines of Rome,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,Nash Ensemble


Terkadang penamaan kolom ataupun tabel pada database cukup panjang dan redundan untuk diketik, ataupun belum cukup informatif.

Penggunaan statement `AS` dapat digunakan untuk melakukan **aliasing / rename** nama tabel maupun nama kolom.

In [14]:
# code here
pd.read_sql_query(
    """
    SELECT AlbumId, Title, Name
    FROM albums AS alb
    LEFT JOIN artists AS ar
    ON alb.ArtistId = ar.ArtistId
    """,
    conn
)

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith
...,...,...,...
342,343,Respighi:Pines of Rome,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,Nash Ensemble


**Note**: Ketika nama tabel sudah dialiasing, harus menggunakan nama alias tersebut di seluruh query

> ⚠️ Hati-hati! Saat melakukan `LEFT JOIN`, peletakkan tabel kiri dan kanan sangat berpengaruh terhadap hasil penggabungan.

In [15]:
pd.read_sql_query(
    """
    SELECT albums.*, artists.Name, GenreId
    FROM albums
    LEFT JOIN artists
    ON albums.ArtistId = artists.ArtistId
    LEFT JOIN tracks
    ON albums.AlbumId = tracks.AlbumId
    """,
    conn
)

,AlbumId,Title,ArtistId,Name,GenreId
0,1,For Those About To Rock We Salute You,1,AC/DC,1
1,1,For Those About To Rock We Salute You,1,AC/DC,1
2,1,For Those About To Rock We Salute You,1,AC/DC,1
3,1,For Those About To Rock We Salute You,1,AC/DC,1
4,1,For Those About To Rock We Salute You,1,AC/DC,1
...,...,...,...,...,...
3498,343,Respighi:Pines of Rome,226,Eugene Ormandy,24
3499,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet,24
3500,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",24
3501,346,Mozart: Chamber Music,274,Nash Ensemble,24



**END OF DAY 1**

---

**START OF DAY 2**

### 🤿 Dive Deeper 

Buatlah sebuah object dataframe bernama `df_tracks` yang menampilkan:

- Semua kolom pada tabel `tracks`
- Kolom `Title` pada tabel `albums`
- Kolom `Name` pada tabel `artists`
- Kolom `Name` pada tabel `genres`

Lalu set kolom `TrackId` menjadi index pada dataframe tersebut. Apabila sudah benar, seharusnya dataframe Anda memiliki 3503 baris dan 11 kolom.

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>

> **Hint 1**: Pada kasus ini, Anda wajib menggunakan syntax `<nama_tabel>.<nama_kolom>` pada statement `SELECT`. Hal ini karena terdapat nama kolom yang sama dari tabel yang berbeda. Jangan lupa untuk menggunakan aliasing `AS` agar nama kolom akhir tidak ambigu.
    
> **Hint 2**: Anda dapat menggunakan `SELECT tracks.*` untuk mengambil semua kolom pada tabel `tracks`

> **Hint 3**: Anda dapat melanjutkan hasil operasi `LEFT JOIN` dengan `LEFT JOIN` berikutnya. Sebagai referensi, berikut adalah struktur kodenya:
    
```python
___ = pd.read_sql_query(
    """
    SELECT ___
    FROM ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    """, conn, index_col=___)
```
</details>

- Jumlah table yang akan di join ada **4**, yaitu : 
1. Tracks
2. Albums
3. Artists
4. Genre

- Kolom yang akan dipilih :
1. Semua kolom pada table Tracks
2. Title -> Albums
3. Name -> Artists
4. Name -> Genres

- Table sisi kiri : `Tracks`

In [16]:
# your code here
df_tracks = pd.read_sql_query(
    """
    SELECT tracks.*, Title, artists.Name AS ArtistName, genres.Name AS GenreName
    FROM tracks
    LEFT JOIN albums
    ON tracks.AlbumId = albums.AlbumId
    LEFT JOIN artists
    ON albums.ArtistId = artists.ArtistId
    LEFT JOIN genres
    ON tracks.GenreId = genres.GenreId
    """, conn, index_col='TrackId'
)

df_tracks

,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,ArtistName,GenreName
TrackId,,,,,,,,,,,
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,AC/DC,Rock
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,Accept,Rock
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Restless and Wild,Accept,Rock
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Restless and Wild,Accept,Rock
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Restless and Wild,Accept,Rock
...,...,...,...,...,...,...,...,...,...,...,...
3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,Respighi:Pines of Rome,Eugene Ormandy,Classical
3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Classical
3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Classical


Gunakan object dataframe `df_tracks` yang telah Anda peroleh di atas dalam menjawab pertanyaan berikut:

1. Pilihlah semua nama genre yang terdapat pada 5 baris terakhir pada dataframe `df_tracks`. Jawaban bisa lebih dari satu.
    - [ ] Latin
    - [x] Classical
    - [x] Soundtrack
    - [ ] Pop

In [17]:
# your code here
df_tracks.tail()

,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,ArtistName,GenreName
TrackId,,,,,,,,,,,
3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,Respighi:Pines of Rome,Eugene Ormandy,Classical
3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Classical
3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Classical
3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,Mozart: Chamber Music,Nash Ensemble,Classical
3503,Koyaanisqatsi,347,2,10,Philip Glass,206005,3305164,0.99,Koyaanisqatsi (Soundtrack from the Motion Pict...,Philip Glass Ensemble,Soundtrack


2. Di antara genre berikut, manakah yang **termasuk** dalam top 3 genre yang paling banyak muncul pada dataframe `df_tracks`? Buatlah sebuah tabel frekuensi dengan metode yang telah Anda pelajari!

    - [x] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>

Beberapa pilihan method yang dapat Anda gunakan:
    
- `.value_counts()`
- `pd.crosstab(index=___, columns=___)`
- `.pivot_table(index=___, aggfunc='count')`
- `.groupby().count()`
</details>

In [18]:
# your code here
df_tracks['GenreName'].value_counts().head(3)

GenreName
Rock     1297
Latin     579
Metal     374
Name: count, dtype: int64

3. Gunakan `groupby()` pada kolom `ArtistName` untuk menghitung `mean()` dari `UnitPrice`. Anda akan melihat bahwa mayoritas artists memiliki rata-rata `UnitPrice` di angka 0.99, namun ada juga beberapa artists yang rata-ratanya 1.99. Dari nama artists berikut, pilihlah yang memiliki rata-rata `UnitPrice` sebesar 1.99:

    - [x] The Office
    - [ ] Red Hot Chili Peppers
    - [ ] Frank Sinatra	
    - [ ] Aerosmith

In [19]:
# your code here
PriceArtist = df_tracks.groupby('ArtistName').agg({'UnitPrice':'mean'})
PriceArtist

,UnitPrice
ArtistName,
AC/DC,0.99
Aaron Copland & London Symphony Orchestra,0.99
Aaron Goldberg,0.99
Academy of St. Martin in the Fields & Sir Neville Marriner,0.99
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,0.99
...,...
Vinícius De Moraes,0.99
Wilhelm Kempff,0.99
Yehudi Menuhin,0.99


In [20]:
PriceArtist[PriceArtist['UnitPrice'] == 1.99]

,UnitPrice
ArtistName,
Aquaman,1.99
Battlestar Galactica (Classic),1.99
The Office,1.99


## SQL Aggregation

Di `pandas`, kita bisa menggunakan ketiga method berikut untuk membuat tabel agregasi:

- `.crosstab()`
- `.pivot_table()`
- `.groupby()`

Sedangkan di SQL, kita menggunakan statement `GROUP BY` yang diletakkan setelah `SELECT ... FROM ...`.

Misal kita ingin mengetahui top 5 `CustomerId` berdasarkan jumlah besaran transaksinya (`Total`) dan juga tampilkan banyak transaksinya:

In [21]:
top_cust = pd.read_sql_query("SELECT CustomerId, SUM(Total), \
                              COUNT(CustomerId) \
                              FROM invoices \
                              GROUP BY CustomerId \
                              ORDER BY SUM(Total) DESC \
                              LIMIT 5", conn, index_col='CustomerId')
top_cust

,SUM(Total),COUNT(CustomerId)
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


In [22]:
# re-create the table above
pd.read_sql_query(
    """
    SELECT CustomerId, SUM(Total), COUNT(CustomerId)
    FROM invoices
    GROUP BY CustomerId
    ORDER BY SUM(Total) DESC
    LIMIT 5
    """, conn
)

,CustomerId,SUM(Total),COUNT(CustomerId)
0,6,49.62,7
1,26,47.62,7
2,57,46.62,7
3,45,45.62,7
4,46,45.62,7


Perhatikan bagaimana query di atas dapat mengambil 5 customer teratas dari tabel `invoices`:

1. Data dikelompokkan berdasarkan `CustomerId` dengan statement `GROUP BY`
2. Terdapat dua fungsi agregasi yang digunakan: `SUM()` dan `COUNT()`, masing-masing mengagregasikan kolom `Total` dan `CustomerId`. Berikut adalah fungsi agregasi yang sering digunakan: `SUM`,` AVG`, `COUNT`, `MIN`, dan `MAX`.
3. Statement `ORDER BY` ditambahkan untuk mengurutkan tabel berdasarkan kolom `SUM(Total)` secara `DESC`.

**Note**:

- SQL Statement `GROUP BY` ekuivalen dengan `.groupby()` pada `pandas`
- SQL Statement `ORDER BY` ekuivalen dengan `.sort_values()` pada `pandas`

### 🤿 Dive Deeper

Seorang produser musik ingin menciptakan lagu dengan mempertimbangkan genre yang akan laku di market. Sebelum mulai, ia terlebih dahulu melakukan riset tentang genre musik apa yang sedang hits saat ini. Sebagai seorang data analyst, Anda diminta untuk membantu produser musik dalam membuat keputusan yang tepat.

**Pertanyaan:** Tampilkan top 5 genre musik dengan jumlah lagu terjual paling banyak, sertakan juga dengan total quantity & price yang terjual.

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
Berikut adalah code yang dapat Anda gunakan untuk mengetahui `genres` paling populer dari semua _item_ (lagu) yang terjual.

```python
top_genre = pd.read_sql_query(
    """
    SELECT
    genres.GenreId,
    genres.Name,
    ___(invoice_items.Quantity),
    ___(invoice_items.UnitPrice)
    FROM ___
    LEFT JOIN ___ ON ___
    LEFT JOIN ___ ON ___
    GROUP BY ___
    ORDER BY ___
    """,
    con=conn,
    index_col='GenreId'
```
</details>

- Table yang digunakan : 
1. Genre
2. Tracks
3. Invoice_items

- Penghubung antar table
1. Genre -> Traks = genreId
2. Track -> Invoice_items = trackId


In [23]:
# your code here
top_genre = pd.read_sql_query(
    """
    SELECT
    genres.GenreId,
    genres.Name,
    SUM(invoice_items.Quantity) as TQuantity,
    SUM(invoice_items.UnitPrice) as TUnitPrice
    FROM genres
    LEFT JOIN tracks ON genres.GenreId=Tracks.GenreId
    LEFT JOIN invoice_items ON invoice_items.TrackId=Tracks.TrackId 
    GROUP BY genres.Name
    ORDER BY TQuantity DESC
    LIMIT 5
    """,
    con=conn,
    index_col='GenreId')
top_genre

,Name,TQuantity,TUnitPrice
GenreId,,,
1,Rock,835,826.65
7,Latin,386,382.14
3,Metal,264,261.36
4,Alternative & Punk,244,241.56
2,Jazz,80,79.20


💡 Panduan step-by-step:

1. Lakukan operasi `JOIN` tergantung dari kebutuhan kolom yang ingin dianalisis
2. Gunakan `GROUP BY` apabila ingin membuat tabel agregasi
3. Aplikasikan fungsi agregat pada `SELECT`
4. Tambahkan kolom yang relevan untuk ditampilkan pada `SELECT`
5. Urutkan data menggunakan `ORDER BY`

## `WHERE` Statements

Sampai di sini, kita telah mempelajari beberapa statement SQL yang sering digunakan:

- `SELECT` statement
- SQL `JOIN`
- Aliasing
- SQL Aggregation dengan `GROUP BY`

Sekarang, kita akan melihat teknik untuk melakukan **conditional subsetting** atau **filter baris** menggunakan statement `WHERE` yang diikuti dengan **kondisi**.

### Logical Operator

- Kondisi `WHERE` dapat dikombinasikan dengan logical operator: `IS`, `AND`, `OR`, dan `NOT`:
    - `IS` sama seperti notasi `=`
    - `IS NOT` seperti notasi `!=`
    
- Kondisi pada `WHERE` juga mendukung operator matematis seperti >, >=, <, dan <=

**Kasus 1**: kita ingin melakukan analisis terhadap semua data `invoices` yang ada di `BillingCountry` Germany, maka kita bisa menambahkan statement `WHERE` sebagai berikut:

In [24]:
# code here
pd.read_sql_query(
    """ 
    SELECT * 
    FROM invoices
    WHERE BillingCountry IS 'Germany'
    """, conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
5,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
6,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
7,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
8,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
9,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91


**Note**: Berbeda dengan `pandas` yang menggunakan 2 tanda sama dengan (`==`), tanda ekivalen pada SQL menggunakan 1 tanda sama dengan (`=`)


**Kasus 2**: Berlawanan dengan kasus di atas, misal kita ingin analisis semua data `invoices` dimana `BillingCountry` **selain** negara Germany:

In [25]:
# code here
pd.read_sql_query(
    """ 
    SELECT * 
    FROM invoices
    WHERE BillingCountry IS NOT 'Germany'
    """, conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
4,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
...,...,...,...,...,...,...,...,...,...
379,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
380,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
381,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
382,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


🧐 Misalkan kita hanya tertarik untuk menganalisis data yang terjadi di Canada dan USA. Kira-kira bagaimana cara mengimplementasikannya dalam bentuk query SQL?

In [26]:
# code here
pd.read_sql_query(
    """ 
    SELECT * 
    FROM invoices
    WHERE BillingCountry  = 'Canada' OR BillingCountry  = 'USA'
    """, conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98
...,...,...,...,...,...,...,...,...,...
142,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
143,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
144,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
145,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


### `IN` Operator

Operator `IN` memungkinkan kita menentukan beberapa nilai untuk perbandingan. Misalnya, seperti pada kasus sebelumnya, kita ingin mengambil semua data `invoices` yang terjadi di negara (`BillingCountry`) Canada dan juga USA:

In [27]:
# code here
pd.read_sql_query(
    """ 
    SELECT * 
    FROM invoices
    WHERE BillingCountry IN ('Canada','USA')
    """, conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98
...,...,...,...,...,...,...,...,...,...
142,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
143,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
144,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
145,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


### ✅ Knowledge Check


Ambillah seluruh data musik yang memiliki genre `Pop` dan `UnitPrice` dari tracknya adalah 0.99. Kemudian simpan ke object dataframe bernama `popmusic`.

**Pertanyaan:** Ada berapa baris pada dataframe `popmusic`?

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
```python
popmusic = pd.read_sql_query(
    """
    SELECT tracks.*, genres.Name AS GenreName
    FROM ___
    LEFT JOIN ___
    ON ___
    WHERE ___
    """,
    conn,
    index_col='TrackId'
)
```
</details>

In [28]:
# your code here
# Jawaban Pak Stanly
pd.read_sql_query("""
                  SELECT t.name, 
                  g.Name, t.UnitPrice 
                  FROM tracks AS t 
                  JOIN genres AS g ON t.GenreId = g.GenreId 
                  WHERE g.Name = 'Pop' AND t.UnitPrice = 0.99
                  """, conn).head()


,Name,Name,UnitPrice
0,"Dig-Dig, Lambe-Lambe (Ao Vivo)",Pop,0.99
1,Pererê,Pop,0.99
2,TriboTchan,Pop,0.99
3,"Tapa Aqui, Descobre Ali",Pop,0.99
4,Daniela,Pop,0.99


In [29]:
popmusic= pd.read_sql_query(
    """
    SELECT tracks.*,genres.Name AS GenreName
    FROM tracks
    LEFT JOIN genres
    ON genres.GenreId = tracks.GenreId
    WHERE tracks.UnitPrice = 0.99 AND GenreName = 'Pop'
    """,
    conn)
popmusic.head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,GenreName
0,323,"Dig-Dig, Lambe-Lambe (Ao Vivo)",29,1,9,Cassiano Costa/Cintia Maviane/J.F./Lucas Costa,205479,6892516,0.99,Pop
1,324,Pererê,29,1,9,Augusto Conceição/Chiclete Com Banana,198661,6643207,0.99,Pop
2,325,TriboTchan,29,1,9,Cal Adan/Paulo Levi,194194,6507950,0.99,Pop
3,326,"Tapa Aqui, Descobre Ali",29,1,9,Paulo Levi/W. Rangel,188630,6327391,0.99,Pop
4,327,Daniela,29,1,9,Jorge Cardoso/Pierre Onasis,230791,7748006,0.99,Pop


**END OF DAY 2**

---

**START OF DAY 3**

### Operating Dates

Mari kita melihat informasi pada table `invoices`

In [30]:
df_invoice = pd.read_sql_query("""SELECT * 
        FROM invoices""", 
        conn)
df_invoice.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


Terdapat kolom `InvoiceDate` yang berisi informasi transaksi pembelian dilakukan. 

Pada operasi kondisi pada statement `WHERE` sebelumnya, kita dapat mengambil semua data `invoices` pada negara Germany. Namun, juga umum untuk kita melakukan conditional query untuk mengambil data pada **rentang tanggal** tertentu.

**Kasus**: Kita ingin meninjau penjualan (`invoices`) di seluruh negara pada tahun 2012 saja.

In [31]:
# your code here
df_invoice_2012 = pd.read_sql_query(
    """  
    SELECT *
    FROM invoices
    WHERE InvoiceDate >= '2012-01-01' AND InvoiceDate <= '2012-12-31'
    """, conn
)
df_invoice_2012

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22 00:00:00,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28 00:00:00,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


In [32]:
df_invoice_2012 = pd.read_sql_query("""SELECT * 
        FROM invoices WHERE strftime('%Y', InvoiceDate) = '2012'""", 
        conn)
df_invoice_2012

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22 00:00:00,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28 00:00:00,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


Cek tipe data pada kolom `InvoiceDate`

In [33]:
# code here
df_invoice_2012.dtypes

InvoiceId              int64
CustomerId             int64
InvoiceDate           object
BillingAddress        object
BillingCity           object
BillingState          object
BillingCountry        object
BillingPostalCode     object
Total                float64
dtype: object

Perhatikan bahwa `InvoiceDate` dikenali sebagai tipe data `object`. 

`pd.read_sql_query()` berperilaku seperti `pd.read_csv()`, yaitu secara default membaca tipe data suatu kolom sebagai numerik dan objek. Untuk mengubah tipe data suatu kolom menjadi `datetime64[ns]` saat pembacaan data, kita dapat menambahkan parameter `parse_dates`

In [34]:
#code here
df_invoice_2012 = pd.read_sql_query(
    """  
    SELECT *
    FROM invoices
    WHERE InvoiceDate >= '2012-01-01' AND InvoiceDate <= '2012-12-31'
    """, conn, parse_dates='InvoiceDate'
)
df_invoice_2012

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


⚠️ **Additional Information:**

Saat melakukan penarikan data pada database dengan menggunakan `pandas`, maka informasi data types akan dilakukan *implicit coercion* (secara otomatis diobah ke dalam bentuk tipe data yang paling general), namun pada skema *database* biasanya struktur tipe data kolom *date* sudah berupa *datetime* data types. 

### `BETWEEN` Operator

Melanjutkan kasus sebelumnya, apabila kita ingin menggunakan kondisi pada rentang tertentu, pendekatan yang umum digunakan adalah operator `BETWEEN`. Silahkan lengkapi code berikut dan lihat apakah data yang terambil sama seperti sebelumnya:

```python
pd.read_sql_query("""SELECT *
        FROM invoices
        WHERE _____ BETWEEN '_____' AND '_____'
        """, 
        con=conn, 
        parse_dates='InvoiceDate')
```

In [35]:
# code here
pd.read_sql_query("""SELECT *
        FROM invoices
        WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-31'
        """, 
        con=conn, 
        parse_dates='InvoiceDate')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


⚠️ **Hati-hati!**

Operator `BETWEEN` sejatinya adalah inclusive, di mana kondisi _start_ dan _end_-nya termasuk. Namun, saat kita membandingkan date (misal '2012-12-31') dengan datetime (pada kolom `InvoiceDate`), seakan-akan end tidak inclusive. 

> `BETWEEN 2 AND 5` mengembalikan nilai 2, 3, 4, 5

In [36]:
# bukti bahwa operator BETWEEN inclusive pada start dan end
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE InvoiceId BETWEEN 2 AND 5
    """,
    con=conn,
    parse_dates='InvoiceDate'
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86


Contoh implementasi pada data tanggal : 

In [37]:
# code here
pd.read_sql_query("""SELECT *
        FROM invoices
        WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-31'
        """, 
        con=conn, 
        parse_dates='InvoiceDate')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


**Info:** Data yang kita miliki merupakan data transaksi dari `2012-01-01` sampai dengan `2012-12-30`

Maka dari itu, sebagai eksperimen, cobalah ganti kondisi pada statement `WHERE` menjadi:

```
WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
```

Maka `InvoiceDate` pada tanggal `2012-12-30` tidak masuk.

In [38]:
# MASALAH: ketika hanya menggunakan '2012-12-30' maka data tanggal 30 Desember 2012 TIDAK masuk
pd.read_sql_query(sql=
        """SELECT *
        FROM invoices
        WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
        """, 
        con=conn, 
        parse_dates='InvoiceDate')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
77,327,1,2012-12-07,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,13.86
78,328,15,2012-12-15,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28,627 Broadway,New York,NY,USA,10012-2612,1.98


Sebagai solusinya, lebih baik kita menambahkan komponen waktu (time) pada kondisi:
```
WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-30 23:59:59'
```

In [39]:
# code here
pd.read_sql_query(sql=
        """SELECT *
        FROM invoices
        WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-30 23:59:59'
        """, 
        con=conn, 
        parse_dates='InvoiceDate')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


### `LIKE` Operator

Operator `LIKE` sangat berguna jika kita perlu mencocokan bagian tertentu dari sebuah string daripada menggunakan operator sama dengan (`=`).

Pada query di bawah ini, `'107%'` ditujukan untuk mengekstrak nilai `BillingPostalCode` yang **dimulai** dengan angka 107. Ini sangat membantu ketika Anda ingin mengekstrak data hanya pada wilayah tertentu.

💡 **Note:** Karakter `%` disebut juga sebagai [wildcard character](https://www.w3schools.com/sql/sql_wildcards.asp)

In [40]:
#code here
pd.read_sql_query(
    """ 
    SELECT *
    FROM invoices
    WHERE BillingPostalCode LIKE '107%'
    """, conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
6,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
7,224,36,2011-09-20 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
8,225,38,2011-09-20 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
9,236,38,2011-10-31 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,13.86


🧐 **Kasus**: Pada 5 data pertama `customerinv`, kita bisa melihat kolom `Company` mungkin tidak dapat diandalkan karena bernilai `None`. Tetapi jika Anda memperhatikan kolom `Email`, Anda dapat melihat beberapa customer memiliki domain email `apple`, yang bisa menjadi indikator perusahaan mereka.

In [41]:
customerinv = pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company, InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    """,
    con=conn)

customerinv.head()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Leonie,Köhler,leonekohler@surfeu.de,None,1,2009-01-01 00:00:00,Germany,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2,2009-01-02 00:00:00,Norway,3.96
2,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
3,Mark,Philips,mphilips12@shaw.ca,Telus,4,2009-01-06 00:00:00,Canada,8.91
4,John,Gordon,johngordon22@yahoo.com,None,5,2009-01-11 00:00:00,USA,13.86


**Pertanyaan:** Bagaimana jika kita ingin mengambil data pembelian dari pelanggan yang memiliki domain email `apple`?

Silahkan lengkapi kode berikut:

```python
applecust = pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company,
        InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices._____ = customers._____
    WHERE _____
    """,
    con=conn
)
```

Berdasarkan query tersebut, berapa pelanggan yang memiliki domain email `apple`?

- [ ] 412
- [ ] 49
- [ ] 7
- [ ] 14

In [42]:
# your code here
pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company, InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    WHERE Email LIKE '%apple%'
    """,
    con=conn)#['Email'].count()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
1,Hugh,O'Reilly,hughoreilly@apple.ie,None,10,2009-02-03 00:00:00,Ireland,5.94
2,Tim,Goyer,tgoyer@apple.com,Apple Inc.,15,2009-03-04 00:00:00,USA,1.98
3,Tim,Goyer,tgoyer@apple.com,Apple Inc.,26,2009-04-14 00:00:00,USA,13.86
4,Terhi,Hämäläinen,terhi.hamalainen@apple.fi,None,53,2009-08-11 00:00:00,Finland,8.91
5,Daan,Peeters,daan_peeters@apple.be,None,55,2009-08-24 00:00:00,Belgium,0.99
6,Hugh,O'Reilly,hughoreilly@apple.ie,None,62,2009-09-24 00:00:00,Ireland,0.99
7,Astrid,Gruber,astrid.gruber@apple.at,None,78,2009-12-08 00:00:00,Austria,1.98
8,Tim,Goyer,tgoyer@apple.com,Apple Inc.,81,2009-12-13 00:00:00,USA,8.91
9,Isabelle,Mercier,isabelle_mercier@apple.fr,None,84,2010-01-08 00:00:00,France,1.98


In [43]:
applecust = pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company,
        InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.customerid = customers.customerid
    WHERE customers.email like '%apple%' GROUP BY email
    """,
    con=conn
)
applecust

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Astrid,Gruber,astrid.gruber@apple.at,None,78,2009-12-08 00:00:00,Austria,1.98
1,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
2,Hugh,O'Reilly,hughoreilly@apple.ie,None,10,2009-02-03 00:00:00,Ireland,5.94
3,Isabelle,Mercier,isabelle_mercier@apple.fr,None,84,2010-01-08 00:00:00,France,1.98
4,Ladislav,Kovács,ladislav_kovacs@apple.hu,None,85,2010-01-08 00:00:00,Hungary,1.98
5,Terhi,Hämäläinen,terhi.hamalainen@apple.fi,None,53,2009-08-11 00:00:00,Finland,8.91
6,Tim,Goyer,tgoyer@apple.com,Apple Inc.,15,2009-03-04 00:00:00,USA,1.98


In [44]:
pd.read_sql_query(
    """
    SELECT Email
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    WHERE Email LIKE '%apple%'
    """,
    con=conn).nunique()

Email    7
dtype: int64

In [45]:
x = pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company, InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    WHERE Email LIKE '%apple%'
    """,
    con=conn)
x['Email'].nunique()

7

In [46]:
applecust = pd.read_sql_query(
    """
    select count(distinct customerId)
    from (
    SELECT invoices.CustomerId, FirstName, LastName, Email, Company,
    InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId 
    WHERE customers.Email like '%apple%') a
    """,conn
)
applecust

,count(distinct customerId)
0,7


## 📝 Summary

### ✅ Knowledge Check 1: SQL Statement Function

Kita telah mempelajari banyak statement pada SQL, silahkan mencocokan statement sesuai dengan kegunaannya:

**KEGUNAAN**

A. Memberikan nama lain pada tabel maupun kolom  
3. `AS <nama_kolom_atau_tabel>`

B. Mengambil beberapa baris teratas dari tabel  
2. `LIMIT <banyaknya_baris>`

C. Mengurutkan baris berdasarkan nilai pada kolom  
6. `ORDER BY <nama_kolom> [ASC|DESC]`

D. Mengambil kolom dari sebuah tabel  
1. `SELECT <nama_kolom> FROM <nama_tabel>`

E. Menggabungkan dua tabel menjadi satu tabel berdasarkan kolom penghubung  
4. `<tabel_kiri> [LEFT|INNER] JOIN <tabel_kanan> ON <tabel>.key = <tabel>.key`

F. Filter baris  
7. `WHERE <kondisi>`

G. Membuat tabel agregasi 
5. `GROUP BY <nama_kolom>` 

**STATEMENT**





















### ✅ Knowledge Check 2: SQL Statement Structure

Susunlah 8 baris SQL statement berikut menjadi sebuah kerangka urutan syntax yang benar:

1. `GROUP BY <nama_kolom>`

2. `LIMIT <banyaknya_baris>`

3. `SELECT <nama_kolom> AS ...`

4. `[LEFT|INNER] JOIN <tabel_kanan> AS ...`

5. `ORDER BY <nama_kolom> [ASC|DESC]`

6. `FROM <nama_tabel> AS ...`

7. `WHERE <kondisi>`
    
8. `ON <tabel>.key = <tabel>.key`

> **JAWABAN**: Urutan yang benar adalah 3-6-4-8-7-1-5-2

---
### 🤿 Dive Deeper

Toko musik digital ini ingin memberikan satu penghargaan ke `employees` yang telah bekerja keras melakukan penjualan di `BillingCountry` Amerika Selatan (yaitu negara **Argentina, Brazil, dan Chile**). Apabila penghargaan tersebut diberikan kepada `employees` berdasarkan **jumlah `Total`** penjualan, siapakah yang berhak mendapatkannya? Tampilkan **nama lengkap (`FirstName` dan `LastName`) beserta total penjualannya**.

Table yang dibutuhkan :
1. Employee
2. Invoices
3. Customer (sebagai table penghubung)

Kolom penghubung : 
1. Employee -> Customer : employees.EmployeesId = customers.SupportRepId
2. Customer -> Employee : customers.CustomerId = invoices.CustomerId

Kondisi : 
- BillingCountry : Argentina, Brazil, Chile

GROUP BY:
- EmployeeId

Aggregate Function : 
- SUM(invoice.Total)

Kolom lain yang ditampilkan :
- FirstName, LastName

In [47]:
# your code here
# Jawaban Dari Pak Aziz
pd.read_sql_query(
    """
    SELECT E.FirstName, E.LastName, I.BillingCountry, SUM(I.Total) AS Total
    FROM employees AS E
    LEFT JOIN customers AS C
        ON  E.EmployeeId = C.SupportRepId
    LEFT JOIN invoices AS I
        ON C.CustomerId = I.CustomerID
    WHERE I.BillingCountry IN ('Argentina', 'Brazil', 'Chile')
    GROUP BY E.EmployeeId
    ORDER BY Total Desc
    """,conn
)

,FirstName,LastName,BillingCountry,Total
0,Margaret,Park,Brazil,112.86
1,Steve,Johnson,Brazil,84.24
2,Jane,Peacock,Brazil,77.24



💭 **Diskusi**: Bagaimana jika kita ingin mengatahui negara mana saja (`BillingCountry`) yang menghasilkan total invoices lebih dari 150 dollar dengan menggunakan fungsi `WHERE`?

In [48]:
# your code here
# pd.read_sql_query(
#     """ 
#     SELECT BillingCountry, SUM(Total) AS TotalInvoice
#     FROM invoices
#     GROUP BY BillingCountry
#     WHERE TotalInvoices > 150
#     """, conn
# )

## **`HAVING` Statement**

Dalam SQL, Anda tidak bisa melakukan statment `WHERE` pada data yang ter aggregasi. Untuk melakukan filter pada data yang telah di aggregasi, Anda bisa menggunakan statement `HAVING`.

Mari selesaikan kasus di atas, yaitu mencari `BillingCountry` yang memiliki `Total` pembelian lebih dari 150 dollar pada tabel `invoices`.

In [49]:
# your code here
pd.read_sql_query(
    """ 
    SELECT BillingCountry, SUM(Total) AS TotalInvoice
    FROM invoices
    GROUP BY BillingCountry
    HAVING TotalInvoice > 150
    """, conn
)

,BillingCountry,TotalInvoice
0,Brazil,190.10
1,Canada,303.96
2,France,195.10
3,Germany,156.48
4,USA,523.06


---

# SQL Subquery

Dalam beberapa kasus tertentu, kita ingin melakukan filter baris berdasarkan syarat tertentu dimana nilai-nilai kondisi didapatkan dari hasil query lain. Apakah Anda masih ingat bagaimana kita mengambil semua pelanggan yang memiliki `invoices` dengan Total paling tinggi? (Bagian SQL Aggregation)

In [50]:
# menampilkan top 5 customers dengan total pembelanjaan terbanyak
pd.read_sql_query("""SELECT CustomerId, SUM(Total) AS TotalSpent, COUNT(CustomerId) AS PurchaseCount
        FROM invoices
        GROUP BY CustomerId
        ORDER BY TotalSpent DESC
        LIMIT 5""", 
        con=conn, 
        index_col='CustomerId')

,TotalSpent,PurchaseCount
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


Misalnya dari tabel tersebut kita ingin menarik semua data `invoices` berdasarkan top 5 `customers`. Kita bisa saja menuliskan list `CustomerId` dalam kondisi secara **hard-code** seperti ini:

In [51]:
# menampilkan invoices hanya untuk top 5 customers

top5cust_hardcode = pd.read_sql_query("""SELECT *
        FROM invoices
        WHERE CustomerId IN (6, 26, 57, 45, 46)
        """, 
        con=conn)
top5cust_hardcode

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,46,6,2009-07-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,8.91
1,175,6,2011-02-15 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
2,198,6,2011-05-20 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,3.96
3,220,6,2011-08-22 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,5.94
4,272,6,2012-04-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,0.99
5,393,6,2013-10-03 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
6,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86
7,70,26,2009-11-07 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,1.98
8,93,26,2010-02-09 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,3.96
9,115,26,2010-05-14 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,5.94


Namun cara tersebut tidak disarankan. Bagaimana jika database kita terus bertambah, sehingga bukan lagi `CustomerId` 6, 26, 57, 45, 46 yang merupakan top 5 customers? Maka dari itu, kita perlu menggunakan **subquery** agar query kita lebih **dinamis terhadap perubahan data**.

Untuk itu mari kita persiapkan subquery yang hanya mengembalikan list Top 5 `CustomerId` berdasarkan `Total` pembelian:

In [52]:
# subquery
pd.read_sql_query(
    """ 
    SELECT CustomerId
    FROM invoices
    GROUP BY CustomerId
    ORDER BY SUM(Total) DESC
    LIMIT 5
    """, conn
)

,CustomerId
0,6
1,26
2,57
3,45
4,46


Subquery di atas kita gunakan untuk menggantikan query hard-code, setelah statement `WHERE` menggunakan operator `IN`:

In [53]:
top5cust_subquery = pd.read_sql_query(
    """ 
    SELECT *
    FROM invoices
    WHERE CustomerId IN (
        SELECT CustomerId
        FROM invoices
        GROUP BY CustomerId
        ORDER BY SUM(Total) DESC
        LIMIT 5
    )
    """
    , conn)
top5cust_subquery

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,46,6,2009-07-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,8.91
1,175,6,2011-02-15 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
2,198,6,2011-05-20 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,3.96
3,220,6,2011-08-22 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,5.94
4,272,6,2012-04-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,0.99
5,393,6,2013-10-03 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
6,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86
7,70,26,2009-11-07 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,1.98
8,93,26,2010-02-09 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,3.96
9,115,26,2010-05-14 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,5.94


Memang hasil `top5cust_hardcode` dengan `top5cust_subquery` akan sama persis, namun secara sintaks `top5cust_subquery` jauh lebih baik karena code lebih dinamis.

---
# [OPTIONAL]Under and Over Fetching

Di antara semua tools yang telah kita pelajari untuk menganalisis data, sekarang saatnya kita merenungkan yang mana yang lebih cocok untuk Anda. Untuk meninjau, mari kita ingat kembali apa yang telah kita pelajari:

- Reading flat files (CSV file)
- Data cleansing and wrangling
- Exploratory data analysis tools
- Visual exploratory tools

Kira-kira SQL masuk ke bagian apa? Untuk menjawab hal tersebut, Anda perlu memahami arsitektur client-server.

![](assets/clientserver.png)

Ketika Anda bekerja dengan SQL, kemungkinan besar Anda memiliki database relasional yang disimpan terpusat pada _server_ dan dapat diakses oleh _client_. Pada dasarnya _server_ adalah komputer penyedia data, dan _client_ adalah komputer pengambil data (komputer Anda).

Saat Anda melakukan query, sebenarnya Anda sedang menjalankan perintah untuk mengunduh data ke komputer Anda (_local computer_). Proses pengunduhan ini membutuhkan sumber daya, sehingga pengunduhan perlu dilakukan secara efektif untuk meminimalkan biaya.

- **Over** fetching adalah kondisi ketika kita menarik data dari database **lebih dari** yang dibutuhkan, sehingga membutuhkan biaya dan waktu yang lebih.

- **Under** fetching adalah kondisi sebaliknya, dimana data yang ditarik **kurang dari** yang dibutuhkan, sehingga proses analisis tidak dapat dilakukan secara lengkap.

💭 **Diskusi**

Anda diminta untuk melakukan analisis terhadap semua penjualan lagu (`invoice_items`) genre `Rock` pada tahun 2012. Pertimbangkan pertanyaan berikut:

- Apakah perlu bagi Anda untuk mengunduh semua tabel `tracks` ke komputer lokal? Tidak Perlu untuk mengunduh semua table tracks
- Apakah Anda melakukan filter baris terhadap `tracks` dengan genre `Rock` menggunakan SQL statement `WHERE` atau conditional subsetting `pandas`? Menggunakan klausa WHERE
- Karena kita memerlukan informasi dari beberapa tabel, manakah cara yang lebih nyaman: melakukan query dengan `JOIN` atau melakukan `SELECT` secara terpisah dari database? Menggunakan JOIN

🧐 Coba buat query yang paling optimum menurut Anda:

In [54]:
# your code here
query = """SELECT invoice_items.*, genres.Name, invoices.InvoiceDate
        FROM invoice_items
        LEFT JOIN invoices
        ON invoice_items.InvoiceId = invoices.InvoiceId
        LEFT JOIN tracks
        ON invoice_items.TrackId = tracks.TrackId
        LEFT JOIN genres
        ON tracks.GenreId = genres.GenreId
        WHERE genres.Name = 'Rock'
              AND invoices.InvoiceDate BETWEEN '2012-01-01' AND '2013-01-01'
        """

pd.read_sql_query(sql=query, con=conn)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,Name,InvoiceDate
0,1355,250,1240,0.99,1,Rock,2012-01-01 00:00:00
1,1357,250,1258,0.99,1,Rock,2012-01-01 00:00:00
2,1358,250,1267,0.99,1,Rock,2012-01-01 00:00:00
3,1363,250,1312,0.99,1,Rock,2012-01-01 00:00:00
4,1364,250,1321,0.99,1,Rock,2012-01-01 00:00:00
...,...,...,...,...,...,...,...
159,1780,327,352,0.99,1,Rock,2012-12-07 00:00:00
160,1795,332,419,0.99,1,Rock,2012-12-30 00:00:00
161,1796,332,423,0.99,1,Rock,2012-12-30 00:00:00
162,1797,332,427,0.99,1,Rock,2012-12-30 00:00:00
